In [0]:
# Silver Incremental Load Notebook 시작
dbutils.widgets.text("execution_date", "2024-01-04")
execution_date = dbutils.widgets.get("execution_date")

from pyspark.sql.functions import col, to_date, year, month, dayofmonth, current_timestamp
from pyspark.sql.utils import AnalysisException

# 마켓 목록
markets = ["KRW-BTC", "KRW-ETH", "KRW-XRP"]

for market in markets:
    bronze_table = f"bronze.bronze_{market.lower().replace('-', '_')}"
    silver_table = f"silver.silver_{market.lower().replace('-', '_')}"

    try:
        # Step 1. Bronze에서 해당 날짜만 필터링
        bronze_df = spark.table(bronze_table)
        bronze_filtered = bronze_df.filter(to_date(col("candle_date_time_kst")) == execution_date)

        if bronze_filtered.count() == 0:
            print(f"⚠️ {market} - 해당 날짜({execution_date}) 데이터 없음. Skip.")
            continue

        # Step 2. Silver 변환
        silver_df = bronze_filtered.withColumn("date", to_date(col("candle_date_time_kst"))) \
            .withColumn("volume", col("candle_acc_trade_volume")) \
            .withColumn("value", col("candle_acc_trade_price")) \
            .withColumn("year", year(col("date"))) \
            .withColumn("month", month(col("date"))) \
            .withColumn("day", dayofmonth(col("date"))) \
            .withColumn("load_timestamp", current_timestamp()) \
            .select(
                "market", "date", "year", "month", "day",
                "opening_price", "high_price", "low_price", "trade_price",
                "prev_closing_price", "change_price", "change_rate",
                "volume", "value", "load_timestamp"
            )

        # Step 3. 기존 silver 테이블에서 해당 날짜 삭제 (있으면)
        try:
            is_exist = (
                spark.table(silver_table)
                .filter((col("market") == market) & (col("date") == execution_date))
                .limit(1)
                .count() > 0
            )
        except AnalysisException:
            is_exist = False  # 테이블이 아예 없는 경우

        if is_exist:
            print(f"🧹 {market} - {execution_date} 기존 데이터 삭제")
            spark.sql(
                f"""
                DELETE FROM {silver_table}
                WHERE market = '{market}' AND date = DATE('{execution_date}')
                """
            )

        # Step 4. Append 저장
        print(f"💾 {market} - {execution_date} 데이터 저장 중...")
        silver_df.write.format("delta") \
            .mode("append") \
            .partitionBy("market", "year", "month") \
            .saveAsTable(silver_table)

        print(f"✅ {market} - {execution_date} 저장 완료")

    except Exception as e:
        print(f"❌ {market} 처리 실패: {e}")
